# Módulo 02 – Preparación de Datos
## Trabajo final

In [131]:
# Importar las librerias necesarias
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [132]:
# Realizar solicitud HTTP y obtener el contenido HTML de la página
url = 'https://www.bcentral.cl/contenido/-/detalle/serie-de-datos-bancarios-septiembre-de-2021'
response = requests.get(url)
html = response.content

# Crear objeto BeautifulSoup para analizar el HTML
soup = BeautifulSoup(html, 'html.parser')

# Encontrar todos los elementos <a> que contengan enlaces de descarga de archivos 
# y que tengan la clase 'linkdoc' (la encontramos al inspeccionar manualmente la página)
enlaces= soup.find_all('a', class_ ='linkdoc')

In [133]:
# Iterar sobre los enlaces encontrados
enlaces_xlsx = []
for enlace in enlaces:
    href = enlace.get('href')
   
    # Verificar si el enlace contiene la palabra "xlsx" en su URL y añadirle la dirección del banco (dada la naturaleza del enlace)
    if href and re.search('.*\.xlsx', href):
        enlaces_xlsx.append('https://www.bcentral.cl/'+ href)

# Descargar los archivos
for enlace in enlaces_xlsx:
    # Construir el nombre del archivo que guardaremos aprovechando la posición 
    archivo_nombre = re.search(r"(.{13})\.xlsx", enlace).group(1) + '.xlsx'
    response = requests.get(enlace)   
    # Guardar el archivo
    with open(archivo_nombre, 'wb') as archivo:
        archivo.write(response.content)
    # Confirmar que se descargó el archivo
    print(f"Archivo descargado: {archivo_nombre}")
    

Archivo descargado: sdbcol_092021.xlsx
Archivo descargado: sdbdep_092021.xlsx
Archivo descargado: sdbinv_092021.xlsx


### El archivo de salida debe considerar las siguiente columnas:
- Fecha
- Institución Financiera
- Tipo (colocación, captación, inversión)
- Subtipo
- Valor
- Tipo de moneda

In [182]:
# Leer los archivos descargados y unificarlos
# Para esto asumiremos que el formato de los archivos es siempre el mismo y no cambia con el tiempo

# Crear las listas necesarias para almacenar los DataFrames
df_unificado = pd.DataFrame()
df_excel = pd.DataFrame()

excel_files = ['sdbcol_092021.xlsx','sdbdep_092021.xlsx','sdbinv_092021.xlsx']

# Iterar sobre los archivos 
for excel in excel_files:
    
    # Acceder al archivo excel y sus hojas
    excel_file = pd.ExcelFile(excel)
    sheet_names = excel_file.sheet_names

    # Imprimir el listado de las hojas de cada archivo
    for sheet_name in sheet_names:
        if len(sheet_name) == 4:      
            df_inicial = pd.read_excel(excel, sheet_name=sheet_name)
            
            # Borrar algunas filas y columnas que no corresponden a registros válidos
            df = df_inicial.drop([0,1,2,3])
            df = df.drop(df.columns[0], axis=1)
            
            # Establecer la primera fila como nombre de las columnas y eliminarla como fila
            df = df.set_axis(df.iloc[0], axis=1)
            df = df[1:]
            
            # Renombrar la columna de bancos
            df = df.rename(columns={df.columns[0]: 'Institución Financiera'})
            
            # Renombrar el eje de columnas en donde ahora están las fechas
            df = df.rename_axis('Fecha', axis='columns')
            
            # Despivotear el dataframe para listar todos los valores con sus fechas e instituciones respectivas
            df_despivot = df.melt(id_vars='Institución Financiera', var_name='Fecha', value_name='Valor')
            
            # Los valores de tipo, subtipos y moneda son unicos por hoja de cada excel y se encuentran siempre en la misma posición, 
            # así que podemos obtenerlos del df original
            Subtipo = df_inicial.iloc[1, 1]
            Moneda =  df_inicial.iloc[2, 1]
            # Obtener el tipo del nombre del excel, más adelante asignaremos el nombre correcto
            Tipo = excel.split('_')[0]
            
            df_despivot['Subtipo'] = Subtipo
            df_despivot['Moneda'] = Moneda
            df_despivot['Tipo'] = Tipo
            
            # Unir toda la info de cada hoja en un mismo archivo
            df_excel = pd.concat([df_excel, df_despivot])
    
    # Unir toda la info de todos los excel en un mismo archivo
    df_unificado = pd.concat([df_unificado, df_excel])

# Imprimir el DataFrame resultante para comprobar que todo salió bien
df_unificado

,Institución Financiera,Fecha,Valor,Subtipo,Moneda,Tipo
0,Banco Bice,2008-01-31,1.76361e+06,Colocaciones totales por tipo de crédito,Saldos en millones de pesos,sdbcol
1,Banco BTG Pactual Chile (9),2008-01-31,ND,Colocaciones totales por tipo de crédito,Saldos en millones de pesos,sdbcol
2,Banco Consorcio (1),2008-01-31,33929.7,Colocaciones totales por tipo de crédito,Saldos en millones de pesos,sdbcol
3,Banco de Chile (2),2008-01-31,1.28772e+07,Colocaciones totales por tipo de crédito,Saldos en millones de pesos,sdbcol
4,Banco de Crédito e Inversiones,2008-01-31,7.80818e+06,Colocaciones totales por tipo de crédito,Saldos en millones de pesos,sdbcol
...,...,...,...,...,...,...
4945,Bank of China (16),2021-09-30,0,Resto de inversiones financieras,Saldos en millones de pesos,sdbinv
4946,Total,2021-09-30,17516.5,Resto de inversiones financieras,Saldos en millones de pesos,sdbinv
4947,NaN,2021-09-30,NaN,Resto de inversiones financieras,Saldos en millones de pesos,sdbinv
4948,NaN,2021-09-30,NaN,Resto de inversiones financieras,Saldos en millones de pesos,sdbinv


In [183]:
df_unificado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 312015 entries, 0 to 4949
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Institución Financiera  286770 non-null  object        
 1   Fecha                   312015 non-null  datetime64[ns]
 2   Valor                   272403 non-null  object        
 3   Subtipo                 312015 non-null  object        
 4   Moneda                  312015 non-null  object        
 5   Tipo                    312015 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 16.7+ MB


In [184]:
# Por exploración sabemos que algunas filas de instituciones financieras no están bien,
# así que podemos listarlas y ver cuáles hay que borrar
df_unificado['Institución Financiera'].unique()

array(['Banco Bice', 'Banco BTG Pactual Chile (9)', 'Banco Consorcio (1)',
       'Banco de Chile (2)', 'Banco de Crédito e Inversiones',
       'Banco de la Nación Argentina (18)', 'Banco del Estado de Chile',
       'Banco do Brasil S.A.', 'Banco Falabella', 'Banco Internacional',
       'Banco Itaú Corpbanca (10)', 'Banco Paris (13)',
       'Banco Penta (14)', 'Banco Ripley', 'Banco Santander - Chile',
       'Banco Security', 'Banco Sudamericano (4) (5) (6)',
       'China Construction Bank, Agencia en Chile (11)',
       'Deutsche Bank (Chile) (12)', 'DnB NOR Bank ASA (7) (8)',
       'HSBC Bank (Chile)', 'JP Morgan Chase Bank, N.A.',
       'Rabobank Chile (15)', 'Scotiabank Chile (3) (17)',
       'The Bank of Tokyo - Mitsubishi Ufj. Ltd. (19)',
       'Bank of China (16)', 'Total', nan, 'ND : No existe dato',
       'Conceptualización: Vínculo que remite al documento conceptual de las estadísticas de colocaciones'],
      dtype=object)

In [185]:
# vamos a borrar las filas que no corresponden a bancos
df_unificado = df_unificado.dropna(subset=['Institución Financiera'])
df_unificado = df_unificado.drop(df_unificado.loc[df_unificado['Institución Financiera'] == 'Total'].index)
df_unificado = df_unificado.drop(df_unificado.loc[df_unificado['Institución Financiera'] == 'ND : No existe dato'].index)
df_unificado = df_unificado.drop(df_unificado.loc[df_unificado['Institución Financiera'] == 'Conceptualización: Vínculo que remite al documento conceptual de las estadísticas de colocaciones'].index)

In [186]:
# Comprobar que se borraron
df_unificado['Institución Financiera'].unique()

array(['Banco Bice', 'Banco BTG Pactual Chile (9)', 'Banco Consorcio (1)',
       'Banco de Chile (2)', 'Banco de Crédito e Inversiones',
       'Banco de la Nación Argentina (18)', 'Banco del Estado de Chile',
       'Banco do Brasil S.A.', 'Banco Falabella', 'Banco Internacional',
       'Banco Itaú Corpbanca (10)', 'Banco Paris (13)',
       'Banco Penta (14)', 'Banco Ripley', 'Banco Santander - Chile',
       'Banco Security', 'Banco Sudamericano (4) (5) (6)',
       'China Construction Bank, Agencia en Chile (11)',
       'Deutsche Bank (Chile) (12)', 'DnB NOR Bank ASA (7) (8)',
       'HSBC Bank (Chile)', 'JP Morgan Chase Bank, N.A.',
       'Rabobank Chile (15)', 'Scotiabank Chile (3) (17)',
       'The Bank of Tokyo - Mitsubishi Ufj. Ltd. (19)',
       'Bank of China (16)'], dtype=object)

In [187]:
# Cambiar el tipo según lo solicitado
df_unificado['Tipo'] = df_unificado['Tipo'].replace({'sdbcol': 'colocación', 'sdbdep': 'captación', 'sdbinv': 'inversión'})

In [188]:
# Verificar el cambio
df_unificado['Tipo'].unique()

array(['colocación', 'captación', 'inversión'], dtype=object)

In [189]:
# Explorar el dataframe
df_unificado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226032 entries, 0 to 4945
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Institución Financiera  226032 non-null  object        
 1   Fecha                   226032 non-null  datetime64[ns]
 2   Valor                   226032 non-null  object        
 3   Subtipo                 226032 non-null  object        
 4   Moneda                  226032 non-null  object        
 5   Tipo                    226032 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 12.1+ MB


In [190]:
# Según lo anterior, no existe ningún valor nulo. Pero si nos fijamos bien el campo valor es reconocido como un object
# y si miramos los archivos excel existe esta nota "ND: No existe dato". Así que sí tenemos valores nulos en esa columna
df_unificado = df_unificado.drop(df_unificado.loc[df_unificado['Valor'] == 'ND'].index)

In [191]:
df_unificado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129927 entries, 0 to 4943
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Institución Financiera  129927 non-null  object        
 1   Fecha                   129927 non-null  datetime64[ns]
 2   Valor                   129927 non-null  object        
 3   Subtipo                 129927 non-null  object        
 4   Moneda                  129927 non-null  object        
 5   Tipo                    129927 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 6.9+ MB


In [194]:
# La columna valor sigue siendo un object, así que vamos a transformala a numero
df_unificado['Valor'] = pd.to_numeric(df_unificado['Valor'], errors='coerce')

In [195]:
df_unificado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129927 entries, 0 to 4943
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Institución Financiera  129927 non-null  object        
 1   Fecha                   129927 non-null  datetime64[ns]
 2   Valor                   129927 non-null  float64       
 3   Subtipo                 129927 non-null  object        
 4   Moneda                  129927 non-null  object        
 5   Tipo                    129927 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 6.9+ MB


In [196]:
# Vamos a eliminar el "saldo" en la columna moneda
df_unificado['Moneda'] = df_unificado['Moneda'].replace({'Saldos en millones de pesos': 'millones de pesos', 'Saldos en millones de dólares': 'millones de dólares'})

In [197]:
df_unificado['Moneda'].unique()

array(['millones de pesos', 'millones de dólares'], dtype=object)

In [198]:
# Resetear los index porque a estas alturas están muy desordenados
df_unificado.reset_index(drop=True, inplace=True)

In [199]:
df_unificado

,Institución Financiera,Fecha,Valor,Subtipo,Moneda,Tipo
0,Banco Bice,2008-01-31,1.763607e+06,Colocaciones totales por tipo de crédito,millones de pesos,colocación
1,Banco Consorcio (1),2008-01-31,3.392970e+04,Colocaciones totales por tipo de crédito,millones de pesos,colocación
2,Banco de Chile (2),2008-01-31,1.287716e+07,Colocaciones totales por tipo de crédito,millones de pesos,colocación
3,Banco de Crédito e Inversiones,2008-01-31,7.808180e+06,Colocaciones totales por tipo de crédito,millones de pesos,colocación
4,Banco de la Nación Argentina (18),2008-01-31,7.512129e+03,Colocaciones totales por tipo de crédito,millones de pesos,colocación
...,...,...,...,...,...,...
129922,Banco Internacional,2021-09-30,0.000000e+00,Resto de inversiones financieras,millones de pesos,inversión
129923,Banco Itaú Corpbanca (10),2021-09-30,0.000000e+00,Resto de inversiones financieras,millones de pesos,inversión
129924,Banco Security,2021-09-30,0.000000e+00,Resto de inversiones financieras,millones de pesos,inversión
129925,"China Construction Bank, Agencia en Chile (11)",2021-09-30,0.000000e+00,Resto de inversiones financieras,millones de pesos,inversión


In [200]:
# Vamos a ordenar las columnas según lo solicitado
# Lista de nombres de columnas en el nuevo orden
orden_solicitado = ['Fecha', 'Institución Financiera', 'Tipo', 'Subtipo', 'Valor', 'Moneda']
df_unificado= df_unificado.reindex(columns=orden_solicitado)
df_unificado

,Fecha,Institución Financiera,Tipo,Subtipo,Valor,Moneda
0,2008-01-31,Banco Bice,colocación,Colocaciones totales por tipo de crédito,1.763607e+06,millones de pesos
1,2008-01-31,Banco Consorcio (1),colocación,Colocaciones totales por tipo de crédito,3.392970e+04,millones de pesos
2,2008-01-31,Banco de Chile (2),colocación,Colocaciones totales por tipo de crédito,1.287716e+07,millones de pesos
3,2008-01-31,Banco de Crédito e Inversiones,colocación,Colocaciones totales por tipo de crédito,7.808180e+06,millones de pesos
4,2008-01-31,Banco de la Nación Argentina (18),colocación,Colocaciones totales por tipo de crédito,7.512129e+03,millones de pesos
...,...,...,...,...,...,...
129922,2021-09-30,Banco Internacional,inversión,Resto de inversiones financieras,0.000000e+00,millones de pesos
129923,2021-09-30,Banco Itaú Corpbanca (10),inversión,Resto de inversiones financieras,0.000000e+00,millones de pesos
129924,2021-09-30,Banco Security,inversión,Resto de inversiones financieras,0.000000e+00,millones de pesos
129925,2021-09-30,"China Construction Bank, Agencia en Chile (11)",inversión,Resto de inversiones financieras,0.000000e+00,millones de pesos


In [201]:
# Guardar el dataframe en un archivo CSV
df_unificado.to_csv('sdbtotal_092021.csv', index=False)